In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained("adapters")

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="cuda"
)

model = PeftModel.from_pretrained(base_model, "adapters")
model.eval()

prompt = """### Instruction:
Convert the following text to formal English using professional and academic language.

### Input:
Thanks for your problem.

### Response:
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=False,
        num_beams=1
    )

full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)


response_start = full_response.find("### Response:") + len("### Response:")
response_text = full_response[response_start:].strip()

if "### Response:" in response_text:
    response_text = response_text.split("### Response:")[0].strip()

print(response_text)

I would appreciate assistance with problem.
